In [1]:
!nvidia-smi

Sun May 15 15:49:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
!pip install pygame

     |████████████████████████████████| 21.8 MB 1.4 MB/s 


In [41]:
!pip install --upgrade gym

     |████████████████████████████████| 626 kB 8.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.23.1-py3-none-any.whl size=701378 sha256=38584d9e4af3d504f9f908367fd0fa39a7bd47b2f5d58cffc1c1eb10710e0518
  Stored in directory: /root/.cache/pip/wheels/e3/33/04/6723848e46f0f1ebe794bb329b7c761c3329a0d7ffade99da7
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3


In [1]:
!pip install ray==1.9

# Imports:

In [2]:
from itertools import product
import gym
import torch
import numpy as np 
from src.visualizations import *
from src.DQL.quant_sim import Trainer
import os
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]


## Environment setup:

In [9]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

activation_function = 'sigmoid'
global_seed = None

epochs = 5000
max_steps = 60
non_random_chance = 0.99
window = 40
target_win_ratio = 0.90
min_steps_num = 6

## Helpers:

In [10]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch import linalg as LA

import numpy as np 
from tqdm import tqdm

from src.entropies import entanglement_entropy, classical_entropy
from src.visualizations import *
from src.utils import one_hot, uniform_linear_layer


class Agent(nn.Module):
    def __init__(self, observation_space_size, n_hidden_layers, activation_function):
        super(Agent, self).__init__()
        self.observation_space_size = observation_space_size
        self.hidden_size = 2*self.observation_space_size

        self.l1 = nn.Linear(in_features=2*self.observation_space_size, out_features=self.hidden_size)
        self.hidden_layers = [
            nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size) \
                for i in range(n_hidden_layers)
        ]
        self.l2 = nn.Linear(in_features=self.hidden_size, out_features=32) 
        self.activation = None
        if activation_function=='lrelu':
            self.activation = F.leaky_relu
        if activation_function=='sigmoid':
            self.activation = F.sigmoid
        if activation_function=='tanh':
            self.activation = F.tanh

        uniform_linear_layer(self.l1)
        for l in self.hidden_layers:
            uniform_linear_layer(l)

        uniform_linear_layer(self.l2)
        
        print('Set the neural network with:')
        print(f'\tInput size: \t{2*self.observation_space_size}')
        for i, l in enumerate(range(n_hidden_layers)):
            print(f'\tHidden {i+1}. layer size: \t{self.hidden_size}')
        print(f'\tOutput size: \t{32}')
    
    def forward(self, state):
        obs_emb = one_hot([int(2*state)], 2*self.observation_space_size)
        # first layer:
        out1 = self.activation(self.l1(obs_emb))
        
        # hidden layers:
        for l in self.hidden_layers:
            out1 = self.activation(l(out1))
        
        # output layers:
        out2 = self.activation(self.l2(out1))

        return out2.view((-1)) 


class Trainer:
    def __init__(self, n_hidden_layers, lake, learning_rate, non_random_chance, random_scaling, gamma, activation_function):
        self.holes_indexes = np.array([5,7,11,12])

        self.lake = lake
        self.agent = Agent(self.lake.observation_space.n, n_hidden_layers, activation_function)
        self.optimizer = optim.Adam(params=self.agent.parameters(), lr=learning_rate)
        
        self.epsilon = non_random_chance
        self.epsilon_growth_rate = random_scaling
        self.gamma = gamma
        
        self.epsilon_list = []
        self.success = []
        self.jList = []
        self.reward_list = []

        self.compute_entropy = True
        self.entropies = []
        self.cl_entropies = []
        self.entropies_episodes = [0]
        
        self.print = False

    
    def train(self, epoch, max_steps, window, target_win_ratio):
        # entropies_episodes = [0] * (epoch+1)
        #pbar = tqdm(range(epoch))
        for i in range(epoch):
            #pbar.set_description(f'Success rate: {sum(self.success[-window:])/window:.2%} | Random chance: {self.epsilon:.2%}')
            
            s = self.lake.reset() #stan na jeziorze 0-16, dla resetu 0
            j = 0
            self.entropies_episodes.append(0)
            while j < max_steps:
                j += 1
                # perform chosen action
                a = self.choose_action(s)
                s1, r, d, _ = self.lake.step(int(a))
                if d == True and r == 0: r = -1
                elif d== True: r == 1
                elif r==0: r = -0.01

                # if self.print==False:
                #     print(self.agent(s)[a])
                #     self.print=True

                # calculate target and loss
                target_q = r + self.gamma * torch.max(self.calc_probabilities(s1).detach()) 

                loss = F.smooth_l1_loss(self.calc_probability(s, a), target_q) 
                # update model to optimize Q
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                # update state
                s = s1
                if(self.compute_entropy):
                    self.entropies.append(entanglement_entropy(self.calc_probabilities(s))) 
                    self.cl_entropies.append(classical_entropy(self.calc_probabilities(s))) 
                    self.entropies_episodes[i] += 1
                
                if d == True: break
            
            # append results onto report lists
            if d == True and r > 0:
                self.success.append(1)
            else:
                self.success.append(0)

            self.reward_list.append(r)
            self.jList.append(j)

            if self.epsilon < 1.:
                self.epsilon *= self.epsilon_growth_rate
            self.epsilon_list.append(self.epsilon)

            tune.report(
                if_trained = sum(self.success[-window:])/window>target_win_ratio, 
                win_ratio = sum(self.success[-window:])/window,
                episode_reward_mean = sum(self.reward_list[-window:])/window,
                steps = len(self.success)
                )

            if i%10==0 and i>100:
                if sum(self.success[-window:])/window>target_win_ratio:
                    #print("Network trained before epoch limit on {i} epoch".format(i=i))
                    break

        #print("last 100 epoches success rate: " + str(sum(self.success[-100:])/100) + "%")

    def choose_action(self, s):
        self.calc_probabilities(s)
        if np.random.rand(1) > self.epsilon : 
            action = torch.argmax(self.calc_probabilities(s)) #wybor najwiekszej wartosci z tablicy
        else:
            action = torch.tensor(np.random.randint(0, 4))
        return action
    
    def calc_probability(self, s, a): #liczenie prawdopodobieństwa obsadzenia kubitu (0-3) z danego stanu planszy (0-15)
        raw_wavefunction = torch.complex(self.agent(s)[0::2], self.agent(s)[1::2])
        probabilities = (raw_wavefunction.abs()**2)
        probabilities = probabilities/probabilities.sum() #normowanie
        prob_indexes = [
            [0,1,2,3,4,5,6,7],
            [0,1,2,3,8,9,10,11],
            [0,1,4,5,8,9,12,13],
            [0,2,4,6,8,10,12,14]
        ]
        return probabilities[prob_indexes[a]].sum()

    def calc_probabilities(self, s): #liczenie prawdopodobieństw każdego z kubitów z danego stanu planszy (0-15) do tensora o kształcie (4)
        raw_wavefunction = torch.complex(self.agent(s)[0::2], self.agent(s)[1::2])
        probabilities = (raw_wavefunction.abs()**2)
        probabilities = probabilities/probabilities.sum() #normowanie
        probs_of_qubits = torch.tensor([
            probabilities[[0,1,2,3,4,5,6,7]].sum(),
            probabilities[[0,1,2,3,8,9,10,11]].sum(),
            probabilities[[0,1,4,5,8,9,12,13]].sum(),
            probabilities[[0,2,4,6,8,10,12,14]].sum()
            ])
        return probs_of_qubits

        
    def Q(self):
        Q = []
        for x in range(self.lake.observation_space.n):
            Qstate = self.agent(x).detach()
            Qstate /= LA.norm(Qstate)
            Q.append(Qstate)   
        Q_out = torch.Tensor(self.lake.observation_space.n, self.lake.action_space.n)
        torch.cat(Q, out=Q_out)
        return Q_out
    
    def Qstate(self, state):
        Qstate = self.agent(state).detach()
        Qstate /= LA.norm(Qstate)
        return Qstate
    
    def Qstrategy(self):
        return [torch.argmax(self.calc_probabilities(state)).item() for state in range(self.lake.observation_space.n)]

# Ray wrapper:

In [16]:
def perform_experiment(config):
    # torch.manual_seed(global_seed)
    # np.random.seed(global_seed)

    results_path = os.path.join('.', 'visuals')
    
    if not os.path.exists(results_path):
        os.mkdir(results_path)

    lake = gym.make('FrozenLake-v1', is_slippery=False)
    lake.reset()
    #print(lake.render(mode='ansi'))

    fl = Trainer(config["n_hidden_layers"], lake, config["lr"], non_random_chance, config["random_scaling"], config["gamma"], activation_function)

    print("Train through {epochs} epochs". format(epochs=epochs))
    fl.train(epochs, max_steps, window, target_win_ratio)

    print("Finished Training")

    plot_success_steps_history(fl.jList, fl.success)

    strategy = np.array(fl.Qstrategy()).reshape((4,4))
    strategy_save_path = os.path.join(results_path, "trained_strategy.jpg")
    plot_strategy(strategy, fl.holes_indexes, strategy_save_path)

    entropies = np.array(fl.entropies)
    cl_entropies = np.array(fl.cl_entropies)
    entropies_save_path = os.path.join(results_path, "entropies.jpg")
    plot_entropies(entropies, cl_entropies, entropies_save_path)

    moving_average_history_save_path = os.path.join(results_path, "training_history_moving_average.jpg")
    plot_rolling_window_history(fl.jList, fl.reward_list, fl.success, np.array(fl.epsilon_list), target_win_ratio, min_steps_num, moving_average_history_save_path, window=window)
    history_save_path = os.path.join(results_path, "training_history.jpg")
    plot_history(fl.jList, fl.reward_list, fl.success, np.array(fl.epsilon_list), target_win_ratio, min_steps_num, history_save_path, window=window)



    with open(os.path.join(results_path, "hyperparameters.txt"), "w+") as f:
        f.write(f'gamma;{config["gamma"]}\n')
        f.write(f'epochs;{epochs}\n')
        f.write(f'max_steps;{max_steps}\n')
        f.write(f'learning_rate;{config["lr"]}\n')
        f.write(f'non_random_chance;{non_random_chance}\n')
        f.write(f'random_scaling;{config["random_scaling"]}\n')
        f.write(f'window;{window}\n')
        f.write(f'target_win_ratio;{target_win_ratio}\n')
        f.write(f'min_steps_num;{min_steps_num}\n')
        f.write(f'n_hidden_layers;{config["n_hidden_layers"]}\n')
        f.write(f'activation_function;{activation_function}\n')
        f.write(f'global_seed;{global_seed}\n')

    with open(os.path.join(results_path, "entropies.txt"), "w") as f:
        for ent in fl.entropies:
            f.write(str(ent)+";")
            
    with open(os.path.join(results_path, "cl_entropies.txt"), "w") as f:
        for ent in fl.cl_entropies:
            f.write(str(ent)+";")



## Experiments:

## Setup:

In [17]:
config = {
    "n_hidden_layers": tune.choice([1,2]),
    "lr": tune.loguniform(1e-5, 4e-1),
    "random_scaling": tune.loguniform(0.99999, 0.99),
    "gamma": tune.loguniform(10., 0.1)

}
scheduler = ASHAScheduler(
    time_attr='steps',
    metric='episode_reward_mean',
    mode='max',
    max_t=epochs+1,
    grace_period=epochs,
    reduction_factor=2
    )


reporter = CLIReporter(
    metric_columns=["if_trained", "win_ratio", "episode_reward_mean", "steps"])

## Run:

In [ ]:
result = tune.run(
    partial(perform_experiment),
    resources_per_trial={"cpu": 0.1, "gpu": 0.05},
    config=config,
    num_samples=500,
    scheduler=scheduler,
    progress_reporter=reporter,
    local_dir='./results/auto_ml',
    log_to_file=True,
    verbose=1)

2022-05-15 15:52:02,286	WARNING experiment.py:256 -- No name detected on trainable. Using DEFAULT.
2022-05-15 15:52:02,289	INFO registry.py:70 -- Detected unknown callable for trainable. Converting to class.
2022-05-15 15:52:02,324	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2022-05-15 15:52:02 (running for 00:00:00.46)
Memory usage on this node: 1.4/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 5000.000: None
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/6.98 GiB heap, 0.0/3.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /content/results/auto_ml/DEFAULT_2022-05-15_15-52-02
Number of trials: 16/500 (16 PENDING)


== Status ==
Current time: 2022-05-15 15:52:07 (running for 00:00:05.50)
Memory usage on this node: 2.0/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 5000.000: None
Resources requested: 1.9000000000000006/2 CPUs, 0.9500000000000003/1 GPUs, 0.0/6.98 GiB heap, 0.0/3.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /content/results/auto_ml/DEFAULT_2022-05-15_15-52-02
Number of trials: 35/500 (16 PENDING, 19 RUNNING)




(bundle_reservation_check_func pid=7139) /usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
(bundle_reservation_check_func pid=7139)   from numpy.dual import register_func
(bundle_reservation_check_func pid=7140) /usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
(bundle_reservation_check_func pid=7140)   from numpy.dual import register_func
(bundle_reservation_check_func pid=7139) /usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
(bundle_reservation_check_func pid=7139)   supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
(bundle_reservation_check_func pid=7140) /usr/local/lib/python3.7/dist-packages/

== Status ==
Current time: 2022-05-15 15:52:12 (running for 00:00:10.66)
Memory usage on this node: 2.5/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 5000.000: None
Resources requested: 1.9000000000000006/2 CPUs, 0.9500000000000003/1 GPUs, 0.0/6.98 GiB heap, 0.0/3.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /content/results/auto_ml/DEFAULT_2022-05-15_15-52-02
Number of trials: 35/500 (16 PENDING, 19 RUNNING)




(bundle_reservation_check_func pid=7139) /usr/local/lib/python3.7/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(bundle_reservation_check_func pid=7139)   import imp
(bundle_reservation_check_func pid=7140) /usr/local/lib/python3.7/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(bundle_reservation_check_func pid=7140)   import imp
(ImplicitFunc pid=7139) /usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
(ImplicitFunc pid=7139)   warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


...

== Status ==
Current time: 2022-05-15 21:23:48 (running for 05:31:46.65)
Memory usage on this node: 6.3/12.7 GiB
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 5000.000: -0.85
Resources requested: 2.0000000000000004/2 CPUs, 1.0000000000000002/1 GPUs, 0.0/6.98 GiB heap, 0.0/3.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /content/results/auto_ml/DEFAULT_2022-05-15_15-52-02
Number of trials: 49/500 (16 PENDING, 20 RUNNING, 13 TERMINATED)


== Status ==
Current time: 2022-05-15 21:23:54 (running for 05:31:52.01)
Memory usage on this node: 6.3/12.7 GiB
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 5000.000: -0.85
Resources requested: 2.0000000000000004/2 CPUs, 1.0000000000000002/1 GPUs, 0.0/6.98 GiB heap, 0.0/3.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /content/results/auto_ml/DEFAULT_2022-05-15_15-52-02
Number of trials: 49/500 (16 PENDING, 20 RUNNING, 13 TERMINATED)




(pid=9007) /usr/local/lib/python3.7/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
(pid=9007)   import imp
(ImplicitFunc pid=9007) /usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
(ImplicitFunc pid=9007)   warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(ImplicitFunc pid=9007) Set the neural network with:
(ImplicitFunc pid=9007) 	Input size: 	32
(ImplicitFunc pid=9007) 	Hidden 1. layer size: 	32
(ImplicitFunc pid=9007) 	Output size: 	32
(ImplicitFunc pid=9007) Train through 5000 epochs


(ImplicitFunc pid=9007) /usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
(ImplicitFunc pid=9007) Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
(ImplicitFunc pid=9007)   from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
(ImplicitFunc pid=9007) /usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_schur.py:140: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`,

== Status ==
Current time: 2022-05-15 21:24:00 (running for 05:31:57.72)
Memory usage on this node: 6.3/12.7 GiB
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 5000.000: -0.85
Resources requested: 2.0000000000000004/2 CPUs, 1.0000000000000002/1 GPUs, 0.0/6.98 GiB heap, 0.0/3.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /content/results/auto_ml/DEFAULT_2022-05-15_15-52-02
Number of trials: 49/500 (16 PENDING, 20 RUNNING, 13 TERMINATED)


== Status ==
Current time: 2022-05-15 21:24:05 (running for 05:32:02.79)
Memory usage on this node: 6.3/12.7 GiB
Using AsyncHyperBand: num_stopped=4
Bracket: Iter 5000.000: -0.85
Resources requested: 2.0000000000000004/2 CPUs, 1.0000000000000002/1 GPUs, 0.0/6.98 GiB heap, 0.0/3.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /content/results/auto_ml/DEFAULT_2022-05-15_15-52-02
Number of trials: 49/500 (16 PENDING, 20 RUNNING, 13 TERMINATED)


== Status ==
Current time: 2022-05-15 21:24:10 (running for 05:32:07.87)
Memor

In [ ]:
result.dataframe()

## Clean up

In [6]:
# import shutil

# shutil.rmtree('../content/results')
# os.listdir('../content')

['.config', 'src', '.ipynb_checkpoints', 'sample_data']